# Recommend opening up coffee shops based on distance between Free WiFi Spots and Colleges

In [10]:
!pip install folium
!pip install wget


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Get dataset

In [2]:
import os
import wget

data_dir = 'BOSTON_WICKED_WIFI'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

    
model_path_1 = os.path.join(data_dir, 'wicked_free_boston.csv')
model_path_2 = os.path.join(data_dir, 'Colleges_and_Universities.csv')

if os.path.isfile(model_path_1):
    os.remove(model_path_1)
if os.path.isfile(model_path_2):
    os.remove(model_path_2)
if not os.path.isfile(model_path_1):
    wget.download("https://raw.githubusercontent.com/elvinjgalarza/IBM-THINK-BOSTON-2024/main/wicked_free_boston.csv", out=data_dir)
    print(f"Data set downloaded to: {model_path_1}")
if not os.path.isfile(model_path_2):
    wget.download("https://raw.githubusercontent.com/elvinjgalarza/IBM-THINK-BOSTON-2024/main/Colleges_and_Universities.csv", out=data_dir)
    print(f"Data set downloaded to: {model_path_2}")

Data set downloaded to: BOSTON_WICKED_WIFI/wicked_free_boston.csv
Data set downloaded to: BOSTON_WICKED_WIFI/Colleges_and_Universities.csv


### Preprocess data

In [11]:
import pandas as pd
from IPython.display import display


raw_wifi_data = pd.read_csv(model_path_1)
print("Cleaning " + model_path_1 + ":")
# Remove rows with duplicates and keep one, and Remove rows without coordinates
#df_w_coords = raw_wifi_data.drop_duplicates(subset=['neighborhood_id'], keep ='first')
df_w_coords = raw_wifi_data.dropna(subset=['X', 'Y'], how='all')
'''
# Remove rows that are schools
keyword = 'school'
df_w_null = df_w_coords[df_w_coords['landmark'].isnull()]
df_w_o_null = df_w_coords.dropna(subset=['landmark'], how='all')
df_w_o_null_w_school = df_w_o_null[~df_w_o_null['landmark'].str.contains(keyword, case=False)]

df_wifi = pd.concat([df_w_null, df_w_o_null_w_school])
'''
df_wifi = df_w_coords


#display(df_wifi)
print("Cleaned up: " + str(len(raw_wifi_data)-len(df_wifi)) + " rows from the original " + str(len(raw_wifi_data)) + ".")

Cleaning BOSTON_WICKED_WIFI/wicked_free_boston.csv:
Cleaned up: 12 rows from the original 279.


In [12]:
raw_college_data = pd.read_csv(model_path_2)
df_college = raw_college_data[raw_college_data['Latitude'] != 0]
#display(df)
print("Cleaned up: " + str(len(raw_college_data)-len(df_college)) + " rows from the original " + str(len(raw_college_data)) + ".")

Cleaned up: 3 rows from the original 60.


### Display map (validation)

In [15]:
from folium import Map, FeatureGroup

m = folium.Map(location=[42.3601, -71.0589], zoom_start=13)  # Boston coordinates

wifi_group = FeatureGroup(name="Wifi")
for i, row in df_wifi.iterrows():
  latitude = row['Y']
  longitude = row['X']
  marker = folium.Marker([latitude, longitude], tooltip=df_wifi['_id'][i], icon=folium.Icon(color='gray', icon='signal'))  # Create marker for each data point
  wifi_group.add_child(marker)

m.add_child(wifi_group)

college_group = FeatureGroup(name="College")    
for i, row in df_college.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    marker = folium.Marker([latitude, longitude], tooltip=df_college['Name'][i], icon=folium.Icon(color='blue', icon='user'))
    college_group.add_child(marker)

m.add_child(college_group)

m

### Define KMeans Clustering for colleges

In [17]:
from sklearn.cluster import KMeans

In [19]:
num_clusters = 3

kmeans = KMeans(n_clusters=num_clusters, random_state=42)

### Fit KMeans to College Locations

In [20]:
kmeans.fit(df_college[['Latitude', 'Longitude']]) 

college_cluster_labels = kmeans.labels_

/opt/app-root/lib64/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


### Calculate Cluster Centroids

In [21]:
cluster_centroids = kmeans.cluster_centers_

df_cluster_centroids = pd.DataFrame(cluster_centroids, columns=['Latitude','Longitude'])

display(df_cluster_centroids)

,Latitude,Longitude
0,42.345263,-71.101861
1,42.350215,-71.065594
2,42.302762,-71.155845


In [22]:
m = folium.Map(location=[42.3601, -71.0589], zoom_start=13)  # Boston coordinates

wifi_group = FeatureGroup(name="Wifi")
for i, row in df_wifi.iterrows():
  latitude = row['Y']
  longitude = row['X']
  marker = folium.Marker([latitude, longitude], tooltip=df_wifi['_id'][i], icon=folium.Icon(color='gray', icon='signal'))  # Create marker for each data point
  wifi_group.add_child(marker)

m.add_child(wifi_group)

college_group = FeatureGroup(name="College")    
for i, row in df_college.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    marker = folium.Marker([latitude, longitude], tooltip=df_college['Name'][i], icon=folium.Icon(color='blue', icon='user'))
    college_group.add_child(marker)

m.add_child(college_group)


for i, row in df_cluster_centroids.iterrows():
  lat = row['Latitude']
  lon = row['Longitude']
  folium.CircleMarker(location=[lat, lon], radius=50, color='green').add_to(m)

m

### Define how to compute distance

In [23]:
from geopy.distance import geodesic

In [24]:
max_distance = 1 # defined threshold in miles

def calculate_distance(lat1, lng1, lat2, lng2):
    coords_1 = (lat1, lng1)
    coords_2 = (lat2, lng2)
    return geodesic(coords_1, coords_2).miles

# Initialize an empty list to store minimum distances
min_distances = []

# Loop through each WiFi location
for index, row in df_wifi.iterrows():
  # Initialize minimum distance for the current WiFi location
  min_distance = float('inf')  # Set to a large value
  for lat, lng in df_cluster_centroids[['Latitude', 'Longitude']].itertuples(index=False):
    # Calculate distance from current WiFi location to each college centroid
    distance = calculate_distance(row['Y'], row['X'], lat, lng)
    # Update minimum distance if current distance is smaller
    min_distance = min(min_distance, distance)
  # Append the minimum distance for the current WiFi location
  min_distances.append(min_distance)

# Add a new column 'min_distance_to_college' to df_wifi
df_wifi['min_distance_to_centroid'] = min_distances

#Filter WiFi locations within the threshold distance to colleges
potential_coffee_locations = df_wifi[df_wifi['min_distance_to_centroid'] <= max_distance]

# Print results (these are potential coffee shop locations based on proximity to colleges and free WiFi)
display(potential_coffee_locations)
print("Potential # of Coffee Shop Locations (near Colleges and Free WiFi): " + str(len(potential_coffee_locations)))

/tmp/ipykernel_14061/2185571379.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wifi['min_distance_to_centroid'] = min_distances


,_id,X,Y,neighborhood_id,neighborhood_name,device_serial,device_connectedto,device_address,device_lat,device_long,device_tags,etl_updatedtimestamp,is_current,org1,org2,inside_outside,landmark,ObjectId,min_distance_to_centroid
3,4,-71.063976,42.355989,N_568579452955527921,Parks,Q3AK-CQWK-ZYPY,PARKS-COMMON-AP8,Boston Common,42.355989,-71.063976,[],2024/04/02 07:55:59+00,1,NaN,NaN,NaN,NaN,4,0.407077
12,13,-71.063827,42.355436,N_601230550253961607,Maintenance,Q2CK-N8A5-VD4F,PARKS-COMMONWEST-AP3,"139 Tremont St, Boston, MA 02111",42.355436,-71.063827,['recently-added'],2024/04/02 07:56:06+00,1,NaN,NaN,NaN,NaN,13,0.371541
14,15,-71.054029,42.360320,N_601230550253961310,Downtown Boston - City Hall - Quincy Market,Q3AE-ARNB-HFAK,DT-QUINCYMARKET-AP2,Faneuil Hall Marketplace\nBoston MA 02201,42.360320,-71.054029,[],2024/04/02 07:56:04+00,1,NaN,NaN,NaN,NaN,15,0.914893
16,17,-71.059855,42.360359,N_601230550253961310,Downtown Boston - City Hall - Quincy Market,Q3AE-HJRQ-D68Q,DT-CITYHALL-GUARDSHACK,One City Hall Sq Boston MA 02201,42.360359,-71.059855,[],2024/04/02 07:56:04+00,1,NaN,NaN,NaN,NaN,17,0.759369
17,18,-71.063827,42.355436,N_601230550253961607,Maintenance,Q2CK-MP93-YDN3,PARKS-COMMONEAST-AP1,"139 Tremont St, Boston, MA 02111",42.355436,-71.063827,"['Boston-Common', 'recently-added']",2024/04/02 07:56:06+00,1,NaN,NaN,NaN,Boston Common,18,0.371541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,235,-71.098446,42.332293,N_601230550253961587,BCYF Tremont,Q2FD-5N3B-TTKT,BCYF-4th FL Dep. Comm,"1483 Tremont St, Boston, MA",42.332293,-71.098446,"['BCYF', 'employee', 'Inside']",2024/04/02 07:56:06+00,1,BCYF,NaN,Inside,employee,235,0.912159
235,236,-71.098650,42.332317,N_601230550253961587,BCYF Tremont,Q2FD-4SE4-JW2S,BCYF-3rd FL GED,"1483 Tremont St, Boston, MA",42.332317,-71.098650,"['BCYF', 'Inside', 'employee']",2024/04/02 07:56:06+00,1,BCYF,NaN,Inside,employee,236,0.908601
247,248,-71.064490,42.348350,N_601230550253962329,BCYF Indoor,Q3AC-X96E-VGA8,BCYF-Quincy_CC_AP1,"BPS-Quincy_Elem_232, 885 Washington Street, Bo...",42.348350,-71.064490,"['BCYF', 'BCYF-Grant', 'Inside']",2024/04/02 07:56:08+00,1,BCYF,NaN,Inside,NaN,248,0.140562
255,256,-71.098180,42.332190,N_601230550253961587,BCYF Tremont,Q3AK-EDQ7-X7P4,AgeStrong-1stFL-Tremont-AP1,"1481 Tremont St, Boston, MA",42.332190,-71.098180,['employee'],2024/04/02 07:56:06+00,1,NaN,NaN,NaN,employee,256,0.921815


Potential # of Coffee Shop Locations (near Colleges and Free WiFi): 66


### Enrich data with median income and population density

In [26]:
import numpy as np

population_densities = np.random.randint(5000, 15001, size=len(potential_coffee_locations))
median_incomes = np.random.randint(30000, 150001, size=len(potential_coffee_locations))

# Group DataFrame by neighborhood_id
grouped_data = potential_coffee_locations.groupby('neighborhood_id')
display(grouped_data)

# Sample one value (population density and median income) per neighborhood
# This ensures consistency within neighborhoods
neighborhood_samples = grouped_data[['neighborhood_id']].apply(pd.DataFrame.sample, 1)

# Assign the sampled values back to the original DataFrame (using merge)
potential_coffee_locations = potential_coffee_locations.merge(neighborhood_samples, on='neighborhood_id')

# Rename columns for clarity (optional)
potential_coffee_locations.rename(columns={'population_densities_x': 'population_density', 
                                           'median_incomes_x': 'median_income'}, inplace=True)

display(potential_coffee_locations)

#df_combined = df_wifi.merge(df_demographics, on="neighborhood_id")

#display(df_combined)

ValueError: 'neighborhood_id' is both an index level and a column label, which is ambiguous.

### Perform Random Forest Regression

In [275]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [316]:
factor_weights = [0.6, 0.25, 0.15]


def calculate_suitability_score(min_distance_to_college_cluster, population_density, median_income, factor_weights):
  suitability_score = (factor_weights[0] * min_distance_to_college_cluster +
                     factor_weights[1] * population_density +
                     factor_weights[2] * median_income)
  return suitability_score

df_combined['suitability_score'] = df_combined.apply(lambda row: calculate_suitability_score(
    row['min_distance_to_centroid'], row['population_densities'], row['median_incomes'], factor_weights), axis=1)

display(df_combined)

,_id,X,Y,neighborhood_id,neighborhood_name,device_serial,device_connectedto,device_address,device_lat,device_long,...,is_current,org1,org2,inside_outside,landmark,ObjectId,min_distance_to_centroid,median_incomes,population_densities,suitability_score
0,4,-71.063976,42.355989,N_568579452955527921,Parks,Q3AK-CQWK-ZYPY,PARKS-COMMON-AP8,Boston Common,42.355989,-71.063976,...,1,NaN,NaN,NaN,NaN,4,0.407077,130176,12310,22604.144246
1,13,-71.063827,42.355436,N_601230550253961607,Maintenance,Q2CK-N8A5-VD4F,PARKS-COMMONWEST-AP3,"139 Tremont St, Boston, MA 02111",42.355436,-71.063827,...,1,NaN,NaN,NaN,NaN,13,0.371541,138065,8187,22756.722925
2,15,-71.054029,42.360320,N_601230550253961310,Downtown Boston - City Hall - Quincy Market,Q3AE-ARNB-HFAK,DT-QUINCYMARKET-AP2,Faneuil Hall Marketplace\nBoston MA 02201,42.360320,-71.054029,...,1,NaN,NaN,NaN,NaN,15,0.914893,34175,14364,8717.798936
3,109,-71.098300,42.332240,L_601230550253962684,YEE Tremont,Q3AK-G3NG-D7KN,BCYF_Tremont_YEE_2nd floor-Multi-Purpose-RM-AP5,"1483 Tremont St, Boston, MA",42.332240,-71.098300,...,1,DYEE,NaN,Inside,NaN,109,0.917196,84612,9521,15072.600317
4,79,-71.058141,42.360109,N_601230550253966673,Downtown Boston - City Hall Plaza and Pavilion,Q3AE-TF7U-TX4P,AP-MR86-IWF-ROOM,"One City Hall Sq, Pavilion Bld, IWF Room,\nBos...",42.360109,-71.058141,...,1,NaN,NaN,NaN,City Hall,79,0.782286,42355,9018,8608.219371
5,231,-71.098312,42.332237,N_601230550253961587,BCYF Tremont,Q2CK-UTKT-YZ7P,BCYF-3rd FL Easter Island,"1483 Tremont St, Boston, MA",42.332237,-71.098312,...,1,BCYF,NaN,Inside,employee,231,0.917256,113878,6841,18792.500354


In [250]:
m = folium.Map(location=[42.3601, -71.0589], zoom_start=13)  # Boston coordinates

wifi_group = FeatureGroup(name="Wifi")
for i, row in df_wifi.iterrows():
  latitude = row['Y']
  longitude = row['X']
  marker = folium.Marker([latitude, longitude], tooltip=df_wifi['_id'][i], icon=folium.Icon(color='gray', icon='signal'))  # Create marker for each data point
  wifi_group.add_child(marker)

m.add_child(wifi_group)

college_group = FeatureGroup(name="College")    
for i, row in df_college.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    marker = folium.Marker([latitude, longitude], tooltip=df_college['Name'][i], icon=folium.Icon(color='blue', icon='user'))
    college_group.add_child(marker)

m.add_child(college_group)

cluster_colors = {0: 'white', 1: 'green', 2: 'yellow'} 

for i, row in df_combined.iterrows():
  lat = row['Y']
  lon = row['X']
  cluster = row['cluster']
  folium.CircleMarker(location=[lat, lon], radius=100, color=cluster_colors[cluster], popup=f"Cluster {cluster}").add_to(m)

m